In [1]:
import pandas as pd

In [ ]:
#Clean short_interest
data = pd.read_csv("data/MS8_short_interest.csv", index_col=False)
data = data[["date", "symbolCode", "daysToCoverQuantity", "currentShortPositionQuantity", "averageDailyVolumeQuantity"]].copy()
data.columns = ["date", "ticker", "d2c", "shorts", "volume" ]
sorted = data.sort_values(by=["ticker", "date"])
sorted.to_csv( "data/MS8_short_interest_W.csv", index=False )

In [22]:
mkt_cap = pd.read_csv("./dataR3000/24-25/mktcap_D.csv")
tickers = mkt_cap["ticker"].unique().tolist()
original_tickers = pd.read_csv("../indexes/R3000/24-25.csv")["Ticker"].unique().tolist()
missing_tickers = [ticker for ticker in original_tickers if ticker not in tickers]

In [11]:
#Estraggo capitalizzazione annuale
YEAR = "20-21"
df = pd.read_csv(f"./dataR3000/{YEAR}/mktcap_D.csv")

df['date'] = pd.to_datetime(df['date'])
tickers = sorted(df['ticker'].unique())
years = sorted(df['date'].dt.year.unique())

# costruiamo target_dates
target_dates = pd.DataFrame(
    [(t, pd.Timestamp(y, 4, 30)) for t in tickers for y in years],
    columns=['ticker', 'target_date']
)
target_dates = target_dates.sort_values(['target_date', 'ticker']).reset_index(drop=True)

df_sorted = df.sort_values(['date', 'ticker']).reset_index(drop=True)

# Merge unico con merge_asof
df_merged = pd.merge_asof(
    target_dates,
    df_sorted,
    left_on='target_date',
    right_on='date',
    by='ticker',
    direction='nearest'
)

# distanza in giorni e filtro
df_merged['diff_days'] = (df_merged['date'] - df_merged['target_date']).abs().dt.days
df_filtered = df_merged[df_merged['diff_days'] <= 100].copy()

df_filtered = df_filtered[['date','ticker','marketCap']].sort_values(['ticker','date']).reset_index(drop=True)
df_filtered.to_csv(f"./dataR3000/{YEAR}/mktcap_Y.csv", index=False)

In [55]:
tickers1 = pd.read_csv("./dataR3000/24-25/mktcap_Y.csv")["ticker"].dropna().unique().tolist()
tickers2 = pd.read_csv("./dataR3000/23-24/mktcap_Y.csv")["ticker"].dropna().unique().tolist()
tickers3 = pd.read_csv("./dataR3000/22-23/mktcap_Y.csv")["ticker"].dropna().unique().tolist()
tickers4 = pd.read_csv("./dataR3000/21-22/mktcap_Y.csv")["ticker"].dropna().unique().tolist()
tickers5 = pd.read_csv("./dataR3000/20-21/mktcap_Y.csv")["ticker"].dropna().unique().tolist()

t = tickers1 + tickers2 + tickers3 + tickers4 + tickers5
df = pd.DataFrame(t, columns=["ticker"])
df.to_csv("./dataR3000/all_tickers.csv", index=False)

In [56]:
df1 = pd.read_csv("./dataR3000/24-25/mktcap_Y.csv")
df2 = pd.read_csv("./dataR3000/23-24/mktcap_Y.csv")
df3 = pd.read_csv("./dataR3000/22-23/mktcap_Y.csv")
df4 = pd.read_csv("./dataR3000/21-22/mktcap_Y.csv")
df5 = pd.read_csv("./dataR3000/20-21/mktcap_Y.csv")

df = pd.concat([df1, df2, df3, df4, df5], ignore_index=True)
df.to_csv( "./dataR3000/all_mktcap_Y.csv", index=False )

In [17]:
YEAR = "20-21"
TARGET_YEAR = 2020
MIN_CAP = 100_000_000    # 100 milioni
MAX_CAP = 5_000_000_000  # 5 miliardi

tickers = pd.read_csv(f"../indexes/R3000/{YEAR}.csv")["Ticker"].unique().tolist()
mkt_cap = pd.read_csv(f"./dataR3000/all_mktcap_Y.csv")

In [18]:
mkt_cap["date"] = pd.to_datetime(mkt_cap["date"])
mkt_cap = mkt_cap[mkt_cap["date"].dt.year == TARGET_YEAR]

In [19]:
filtered = mkt_cap[
    (mkt_cap["ticker"].isin(tickers)) &
    (mkt_cap["marketCap"] >= MIN_CAP) &
    (mkt_cap["marketCap"] <= MAX_CAP)
]

In [20]:
filtered = filtered.sort_values("marketCap", ascending=False).reset_index(drop=True)

In [21]:
filtered.to_csv(f"./dataR3000/{YEAR}/filtered_tickers.csv", index=False)

In [66]:
r2000 = pd.read_csv("../indexes/R2000.csv")["Ticker"].unique().tolist()
new_r2000 = filtered["ticker"].unique().tolist()

In [67]:
len(new_r2000)

2021

In [71]:
diff = [ticker for ticker in r2000 if ticker not in new_r2000]

In [72]:
len(diff)

200